In [1]:
!git clone https://github.com/sorki/python-mnist
!./python-mnist/bin/mnist_get_data.sh
!pip3 install emnist
from emnist import extract_training_samples

Cloning into 'python-mnist'...
remote: Enumerating objects: 246, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 246 (delta 1), reused 2 (delta 0), pack-reused 240 (from 1)
Receiving objects: 100% (246/246), 47.14 KiB | 1.88 MiB/s, done.
Resolving deltas: 100% (119/119), done.
--2025-02-18 11:57:22--  http://yann.lecun.com/exdb/mnist/
Resolving yann.lecun.com (yann.lecun.com)... 15.204.224.156
Connecting to yann.lecun.com (yann.lecun.com)|15.204.224.156|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 756 [text/html]
Saving to: ‘data/index.html.tmp’

index.html.tmp      100%[===================>]     756  --.-KB/s    in 0s      

2025-02-18 11:57:22 (75.6 MB/s) - ‘data/index.html.tmp’ saved [756/756]

Loading robots.txt; please ignore errors.
--2025-02-18 11:57:22--  http://yann.lecun.com/robots.txt
Reusing existing connection to yann.lecun.com:80.
HTTP request sent, awaiting response... 404 Not Foun

In [3]:
# X is the images and y is the labels
X, y = extract_training_samples('letters')
X = X / 255.

# First 60000 instances as training and the next 10000 as testing
X_train, X_test = X[:60000], X[60000:70000]
y_train, y_test = y[:60000], y[60000:70000]

X_train = X_train.reshape(60000,784)
X_test = X_test.reshape(10000,784)

BadZipFile: File is not a zip file

In [ ]:
import matplotlib.pyplot as plt

img_index = 14440 #value determines image shown
img = X_train[img_index]
print("Image Label: " + str(chr(y_train[img_index]+96)))
plt.imshow(img.reshape((28,28)))

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.neural_network import MLPClassifier

#MLP, 6 layers, 300 neurons, iterates 50 times
mlp2 = MLPClassifier(hidden_layer_sizes=(300,300,300,300,300,300,), max_iter=75, alpha=1e-4,
                    solver='sgd', verbose=10, tol=1e-4, random_state=1,
                    learning_rate_init=.1)
mlp2.fit(X_train, y_train)
print("Training set score: %f" % mlp2.score(X_train, y_train))
print("Test set score: %f" % mlp2.score(X_test, y_test))

In [ ]:
y_pred = mlp2.predict(X_test) #list of predicted values

from sklearn.metrics import confusion_matrix #errors
cm = confusion_matrix(y_test, y_pred)
plt.matshow(cm)

In [ ]:
#possible confused letters
predicted_letter = 'l'
actual_letter = 'i'

mistake_list = []
for i in range(len(y_test)):
  if (y_test[i] == (ord(actual_letter) - 96) and y_pred[i] == (ord(predicted_letter) - 96)):
    mistake_list.append(i)
print("There were " + str(len(mistake_list)) + " times that the letter " + actual_letter + " was predicted to be the letter " + predicted_letter + ".")

mistake_to_show = 4 #mistake number

if (len(mistake_list)> mistake_to_show): #shows numbered mistake
  img = X_test[mistake_list[mistake_to_show]]
  plt.imshow(img.reshape((28,28)))
else:
  print("Couldn't show mistake number " + str(mistake_to_show + 1) + " because there were only " + str(len(mistake_list)) + " mistakes to show!")

In [ ]:
!git clone https://github.com/ramenndls/my_writing_mod.git #all images are uploaded in this repository, and all have been color inverted
!git pull
!ls my_writing_mod
!cd /content/my_writing_mod
!pwd

import os
path, dirs, files = next(os.walk("/content/my_writing_mod/"))
files.sort()

In [ ]:
import cv2
import matplotlib.pyplot as plt

handwritten_words = [] #adds all files to a list
for i in range(len(files)):
  img = cv2.imread("/content/my_writing_mod/"+files[i],cv2.IMREAD_GRAYSCALE)
  handwritten_words.append(img)

In [ ]:
import numpy
import cv2

words = ""
for letter in handwritten_words:
    letter = cv2.resize(letter, (28,28), interpolation = cv2.INTER_CUBIC)
    single_item_array = (numpy.array(letter)).reshape(1,784)
    prediction = mlp2.predict(single_item_array)
    words = words + str(chr(prediction[0]+96))

print("Conversion complete!")
print(words)

In [ ]:
import numpy

#checks to see if the image is just a blank space
total_pixel_value = 0
for j in range(28):
  for k in range(28):
    total_pixel_value += letter[j,k]
if total_pixel_value < 20:
  words = words + " "
else:         #if it NOT a blank, it actually runs the prediction algorithm on it
  single_item_array = (numpy.array(letter)).reshape(1,784)
  prediction = mlp2.predict(single_item_array)
  words = words + str(chr(prediction[0]+96))

print("Conversion complete!")
print(words)

In [ ]:
processed_words = []

for img in handwritten_words:
  #Apply Gaussian blur filter
  img = cv2.GaussianBlur(img, (7,7), 0)

  #Extract the Region of Interest in the image and center in square
  points = cv2.findNonZero(img)
  x, y, w, h = cv2.boundingRect(points)
  if (w > 0 and h > 0):
    if w > h:
      y = y - (w-h)//2
      img = img[y:y+w, x:x+w]
    else:
      x = x - (h-w)//2
      img = img[y:y+h, x:x+h]

  #Resize and resample to be 28 x 28 pixels
  img = cv2.resize(img, (28,28), interpolation = cv2.INTER_CUBIC)

  #Normalize pixels and reshape before adding to the new story array
  img = img/255
  img = img.reshape((28,28))
  processed_words.append(img)

print("Processed the scanned images.")

import matplotlib.pyplot as plt
plt.imshow(processed_words[0])

In [ ]:
import numpy

typed_story = ""
for letter in processed_words:
  #this bit of code checks to see if the image is just a blank space by looking at the color of all the pixels summed
  total_pixel_value = 0
  for j in range(28):
    for k in range(28):
      total_pixel_value += letter[j,k]
  if total_pixel_value < 20:
    typed_story = typed_story + " "
  else:         #if it NOT a blank, it actually runs the prediction algorithm on it
    single_item_array = (numpy.array(letter)).reshape(1,784)
    prediction = mlp2.predict(single_item_array)
    typed_story = typed_story + str(chr(prediction[0]+96))

true_value = 'abklmnopqrstcuvwxyzdefghij'
print("Conversion complete!"'\n') #true value = abklmnopqrstcuvwxyzdefghij
print('conversion:', typed_story)
print('true value:', true_value)

In [ ]:
from difflib import SequenceMatcher
#compares both strings and outputs a % indicating how accurate the converted value is
s = SequenceMatcher(None, typed_story, true_value)
print('percentage similarity:', s.ratio())